In [1]:
from codes.utils import set_ipynb_config, time_it
set_ipynb_config()

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os

In [4]:
path_data = os.getenv('PATH_DATA')
path_data

'data/multi_docs/'

In [5]:
path_persist_db = os.getenv('PATH_PERSIST_DB')

## C. Retrieval Process
**Expected Input:** Query from user

```python
get_query_topic(query) -> topics  
get_filter_criterion(topics) -> filters
retrieve_chunks(query, filters)  # based on filter criterion
rerank_chunks(query, chunks) -> chunks
```
**Expected Output:** List of chunks (or docs)

In [6]:
from codes.file_to_docs import Docs2VectorDb
from codes.retrieve_docs import RetrieveDocs

/opt/miniconda3/envs/suppbot-langchain/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
vector_store_multi = Docs2VectorDb.load_vector_store(path_persist_db)

/opt/miniconda3/envs/suppbot-langchain/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
sources = Docs2VectorDb.sources_from_vdb(vector_store_multi)
sources

{'source': {'data/multi_docs/Luminate Report Builder.docx',
  'data/multi_docs/Luminate Report Builder.pdf',
  'data/multi_docs/marketing.txt',
  'data/multi_docs/qna_table.csv'}}

In [9]:
query = 'How have the toy stores changed over the years?'

In [10]:
metadata_filt = {
    'filter': {
        '$and': [
            {'title': {'$eq':'marketing.txt'}},
            {'data_type': {'$eq':'txt'}},
            # {'data_type': {'$in':['txt', 'dataframe']}},
            # {'topic': {'$eq':'RB, Luminate'}},
            ]
        }
    }

search_kwargs={
        'k': 4,
        'fetch_k': 20,
}


search_kwargs.update(metadata_filt)
print(search_kwargs)

{'k': 4, 'fetch_k': 20, 'filter': {'$and': [{'title': {'$eq': 'marketing.txt'}}, {'data_type': {'$eq': 'txt'}}]}}


In [11]:
docs_retrvd_w_reranking = RetrieveDocs.main(query, vector_store_multi, **search_kwargs)
RetrieveDocs.pprint_docs(docs_retrvd_w_reranking)

------------------------------
Still, a visit to a toy store or the toys section of a grocery store in most places will make you feel that not much has changed in the past few decades.
{'data_type': 'txt', 'topic': 'marketing, toys'}


To this end, Norgaard and Wider analysed 175 television commercials for toys listed as ‘best selling’ for children ages five through eleven years old by the top three toy retailers — Target, Walmart,
{'data_type': 'txt', 'topic': 'marketing, toys'}


figures, and sports equipment. Meanwhile, girls got dolls, dollhouses, beauty sets and… toy kitchen appliances.
{'data_type': 'txt', 'topic': 'marketing, toys'}


freedom to choose their toys?
{'data_type': 'txt', 'topic': 'marketing, toys'}




## To check

- chunk overlap > word size, will the word get split into two chunks
- mmr >> does it already incorporate reranking >> handle it in code